<a href="https://colab.research.google.com/github/ismail-dev1/Fire_Detection_Dissertation/blob/main/SD_Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch, torchvision
print("Torch:", torch.__version__, "Torchvision:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


Torch: 2.8.0+cu126 Torchvision: 0.23.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB


In [2]:
from google.colab import files
up = files.upload()  # choose your zip, e.g. fire_dataset.zip
zip_name = list(up.keys())[0]

!mkdir -p /content/data
!unzip -q "$zip_name" -d /content/data

# If your zip contains the folder fire_dataset/, set this accordingly:
DATA_DIR = "/content/data/fire_dataset"  # adjust if needed
!ls -R "$DATA_DIR" | head -n 40


Saving fire_dataset.zip to fire_dataset.zip
/content/data/fire_dataset:
fire
no_fire

/content/data/fire_dataset/fire:
ComfyUI_00001_ 2.png
ComfyUI_00001_ copy.png
ComfyUI_00001_.png
ComfyUI_00002_ 2.png
ComfyUI_00002_ copy.png
ComfyUI_00002_.png
ComfyUI_00003_ 2.png
ComfyUI_00003_.png
ComfyUI_00004_ 2.png
ComfyUI_00004_ copy.png
ComfyUI_00004_.png
ComfyUI_00005_ 2.png
ComfyUI_00005_.png
ComfyUI_00006_ 2.png
ComfyUI_00006_ copy.png
ComfyUI_00006_.png
ComfyUI_00007_ 2.png
ComfyUI_00007_ copy.png
ComfyUI_00007_.png
ComfyUI_00008_ 2.png
ComfyUI_00008_.png
ComfyUI_00009_ 2.png
ComfyUI_00009_.png
ComfyUI_00010_ 2.png
ComfyUI_00010_.png
ComfyUI_00011_ 2.png
ComfyUI_00011_.png
ComfyUI_00012_ 2.png
ComfyUI_00012_.png
ComfyUI_00013_ 2.png
ComfyUI_00013_.png
ComfyUI_00014_ 2.png
ComfyUI_00014_.png
ComfyUI_00015_ 2.png
ComfyUI_00015_.png


In [3]:
import os, random, shutil
from collections import defaultdict

SRC = DATA_DIR  # fire/ and no_fire/ live directly under SRC
DST = "/content/data_split"
VAL_RATIO = 0.15
random.seed(42)

classes = [d for d in os.listdir(SRC) if os.path.isdir(os.path.join(SRC,d))]
assert set(classes) >= {"fire","no_fire"}, f"Found classes: {classes}"

for split in ["train","val"]:
    for c in classes:
        os.makedirs(os.path.join(DST, split, c), exist_ok=True)

by_class = {c: [] for c in classes}
for c in classes:
    cdir = os.path.join(SRC, c)
    for f in sorted(os.listdir(cdir)):
        p = os.path.join(cdir, f)
        if os.path.isfile(p):
            by_class[c].append(p)

for c, items in by_class.items():
    random.shuffle(items)
    n_val = max(1, int(len(items)*VAL_RATIO))
    val_items = items[:n_val]
    trn_items = items[n_val:]
    for p in trn_items:
        shutil.copy2(p, os.path.join(DST, "train", c, os.path.basename(p)))
    for p in val_items:
        shutil.copy2(p, os.path.join(DST, "val", c, os.path.basename(p)))

DATA_DIR = DST  # from now on we use the split dataset
print("Split done at", DATA_DIR)
!find "$DATA_DIR" -maxdepth 2 -type d -print


Split done at /content/data_split
/content/data_split
/content/data_split/train
/content/data_split/train/no_fire
/content/data_split/train/fire
/content/data_split/val
/content/data_split/val/no_fire
/content/data_split/val/fire


In [4]:
import os, json, time, math, torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet18, resnet50, ResNet18_Weights, ResNet50_Weights

# ==== paths ====
USE_DRIVE = True   # set False if you don't want to save to Drive
if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    SAVE_DIR = "/content/drive/MyDrive/fire_resnet"
else:
    SAVE_DIR = "/content/runs_fire_resnet"
os.makedirs(SAVE_DIR, exist_ok=True)

# ==== config ====
IMG_SIZE     = 224
BATCH_SIZE   = 64              # bump up since CUDA is faster; reduce if OOM
EPOCHS       = 15
LR           = 3e-4
ARCH         = "resnet18"      # or "resnet50"
PRETRAINED   = True
VAL_SUBDIR   = "val"           # expected subfolders train/ and val/
TRAIN_SUBDIR = "train"
MIXED_PREC   = True            # AMP on CUDA

# ==== data ====
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

train_tf = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
val_tf = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.14)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_dir = os.path.join(DATA_DIR, TRAIN_SUBDIR)
val_dir   = os.path.join(DATA_DIR, VAL_SUBDIR)

train_ds = datasets.ImageFolder(train_dir, transform=train_tf)
val_ds   = datasets.ImageFolder(val_dir,   transform=val_tf)
class_to_idx = train_ds.class_to_idx
idx_to_class = {v:k for k,v in class_to_idx.items()}
print("Classes:", class_to_idx)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

# ==== model ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

if ARCH == "resnet50":
    model = resnet50(weights=ResNet50_Weights.DEFAULT if PRETRAINED else None)
    in_feats = model.fc.in_features
else:
    model = resnet18(weights=ResNet18_Weights.DEFAULT if PRETRAINED else None)
    in_feats = model.fc.in_features

model.fc = nn.Linear(in_feats, len(class_to_idx))
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda" and MIXED_PREC))

def accuracy(logits, y):
    return (logits.argmax(1) == y).float().mean().item()

def run_epoch(loader, train=True):
    model.train(mode=train)
    tot_loss = tot_acc = n = 0
    for xb, yb in loader:
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        if train:
            optimizer.zero_grad(set_to_none=True)
            if scaler.is_enabled():
                with torch.cuda.amp.autocast():
                    out = model(xb)
                    loss = criterion(out, yb)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                out = model(xb); loss = criterion(out, yb)
                loss.backward(); optimizer.step()
        else:
            with torch.no_grad():
                out = model(xb); loss = criterion(out, yb)
        bs = xb.size(0)
        tot_loss += loss.item()*bs
        tot_acc  += accuracy(out, yb)*bs
        n += bs
    return tot_loss/n, tot_acc/n

best_val = 0.0
for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    tr_loss, tr_acc = run_epoch(train_loader, train=True)
    va_loss, va_acc = run_epoch(val_loader,   train=False)
    dt = time.time() - t0
    print(f"Epoch {epoch:02d}/{EPOCHS} | train {tr_loss:.4f}/{tr_acc:.3f} | val {va_loss:.4f}/{va_acc:.3f} | {dt:.1f}s")

    # save per-epoch checkpoint (for resume)
    ckpt_epoch_path = os.path.join(SAVE_DIR, f"ckpt_epoch_{epoch:02d}.pt")
    torch.save({
        "state_dict": model.state_dict(),
        "arch": ARCH,
        "img_size": IMG_SIZE,
        "class_to_idx": class_to_idx,
        "norm": {"mean": mean, "std": std},
        "epoch": epoch,
        "val_acc": va_acc,
    }, ckpt_epoch_path)

    # save best
    if va_acc > best_val:
        best_val = va_acc
        best_path = os.path.join(SAVE_DIR, "best_fire_resnet.pt")
        torch.save({
            "state_dict": model.state_dict(),
            "arch": ARCH,
            "img_size": IMG_SIZE,
            "class_to_idx": class_to_idx,
            "norm": {"mean": mean, "std": std},
            "epoch": epoch,
            "val_acc": va_acc,
        }, best_path)
        with open(os.path.join(SAVE_DIR, "class_to_idx.json"), "w") as f:
            json.dump(class_to_idx, f, indent=2)
        print(f"  Saved new best to {best_path} (val_acc={best_val:.3f})")

print("Training finished. Best val_acc:", best_val)


Mounted at /content/drive
Classes: {'fire': 0, 'no_fire': 1}
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 104MB/s]
/tmp/ipython-input-785717094.py:75: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device.type=="cuda" and MIXED_PREC))
/tmp/ipython-input-785717094.py:88: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch 01/15 | train 0.4476/0.791 | val 0.7668/0.856 | 39.8s
  Saved new best to /content/drive/MyDrive/fire_resnet/best_fire_resnet.pt (val_acc=0.856)
Epoch 02/15 | train 0.3181/0.857 | val 0.4480/0.848 | 17.6s
Epoch 03/15 | train 0.2562/0.900 | val 0.4279/0.860 | 17.4s
  Saved new best to /content/drive/MyDrive/fire_resnet/best_fire_resnet.pt (val_acc=0.860)
Epoch 04/15 | train 0.2030/0.915 | val 0.4300/0.856 | 17.4s
Epoch 05/15 | train 0.1717/0.934 | val 0.4939/0.868 | 17.4s
  Saved new best to /content/drive/MyDrive/fire_resnet/best_fire_resnet.pt (val_acc=0.868)
Epoch 06/15 | train 0.1492/0.943 | val 0.4992/0.833 | 17.3s
Epoch 07/15 | train 0.1046/0.967 | val 0.6025/0.852 | 17.3s
Epoch 08/15 | train 0.0830/0.969 | val 0.6833/0.837 | 17.4s
Epoch 09/15 | train 0.1006/0.964 | val 0.6557/0.798 | 17.3s
Epoch 10/15 | train 0.0939/0.966 | val 0.6040/0.848 | 17.4s
Epoch 11/15 | train 0.0915/0.967 | val 0.5489/0.852 | 17.3s
Epoch 12/15 | train 0.0857/0.965 | val 0.4677/0.844 | 17.4s
Epoch 1

In [5]:
import os, json, torch, numpy as np
from torchvision import datasets, transforms
from torchvision.models import resnet18, resnet50
from torch.utils.data import DataLoader

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BEST_PATH = "/content/drive/MyDrive/fire_resnet/best_fire_resnet.pt"  # adjust if needed
DATA_DIR  = "/content/data_split"  # or your train/val root

# Transforms must match training normalization
IMG_SIZE = 224
mean = [0.485, 0.456, 0.406]; std = [0.229, 0.224, 0.225]
val_tf = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.14)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

val_ds = datasets.ImageFolder(os.path.join(DATA_DIR, "val"), transform=val_tf)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)
class_to_idx = val_ds.class_to_idx
idx_to_class = {v:k for k,v in class_to_idx.items()}

ckpt = torch.load(BEST_PATH, map_location=DEVICE)
arch = ckpt.get("arch", "resnet18")
num_classes = len(ckpt["class_to_idx"])

if arch == "resnet50":
    model = resnet50(num_classes=num_classes)
else:
    model = resnet18(num_classes=num_classes)
model.load_state_dict(ckpt["state_dict"]); model.to(DEVICE).eval()

# Gather predictions
all_probs, all_true, all_paths = [], [], []
with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(DEVICE)
        logits = model(xb)
        probs = torch.softmax(logits, dim=1).cpu().numpy()
        all_probs.append(probs); all_true.append(yb.numpy())
        # stash file paths for error analysis
        start = len(all_paths)
        all_paths.extend([val_ds.samples[start+i][0] for i in range(probs.shape[0])])
all_probs = np.concatenate(all_probs, 0)
all_true  = np.concatenate(all_true, 0)

import csv
os.makedirs("/content/eval", exist_ok=True)
with open("/content/eval/val_predictions.csv","w",newline="") as fh:
    w=csv.writer(fh); w.writerow(["path","true","pred","p_fire"])
    fire_idx = class_to_idx.get("fire", 1)  # fallback in case
    for i in range(len(all_true)):
        pred = int(all_probs[i].argmax())
        p_fire = float(all_probs[i, fire_idx])
        w.writerow([all_paths[i], idx_to_class[all_true[i]], idx_to_class[pred], f"{p_fire:.4f}"])
print("Saved /content/eval/val_predictions.csv")


Saved /content/eval/val_predictions.csv


In [6]:
import numpy as np

n = len(class_to_idx)
cm = np.zeros((n,n), dtype=int)
pred = all_probs.argmax(1)
for t,p in zip(all_true, pred): cm[t,p]+=1
print("Confusion (rows=true, cols=pred):\n", cm)

tp = np.diag(cm).astype(float)
prec = tp / np.maximum(cm.sum(0), 1)
rec  = tp / np.maximum(cm.sum(1), 1)
for i in range(n):
    print(f"{idx_to_class[i]:8s}  precision={prec[i]:.3f}  recall={rec[i]:.3f}")


Confusion (rows=true, cols=pred):
 [[163   9]
 [ 25  60]]
fire      precision=0.867  recall=0.948
no_fire   precision=0.870  recall=0.706


In [8]:
from torchvision import transforms

train_tf = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(12),
    transforms.ColorJitter(0.35,0.35,0.35,0.10),
    transforms.RandomGrayscale(p=0.2),           # de-emphasize color shortcuts
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
    transforms.RandomErasing(p=0.25, scale=(0.02,0.15), ratio=(0.3,3.3)),
])



In [9]:
#RN18 @320 (8-10)epochs
ARCH = "resnet18"
IMG_SIZE = 320
BASE_BS = 64         # what worked at 224
BATCH_SIZE = max(16, int(BASE_BS * (224/IMG_SIZE)**2))  # ≈32 at 320


In [11]:
ACCUM_STEPS = 2
optimizer.zero_grad(set_to_none=True)
# Ensure the model is on the correct device
model.to(device)
for step, (xb, yb) in enumerate(train_loader, 1):
    xb, yb = xb.to(device), yb.to(device)
    if scaler.is_enabled():
        with torch.amp.autocast('cuda'):
            out = model(xb)
            loss = criterion(out, yb) / ACCUM_STEPS
        scaler.scale(loss).backward()
        if step % ACCUM_STEPS == 0:
            scaler.step(optimizer); scaler.update()
            optimizer.zero_grad(set_to_none=True)
    else:
        out = model(xb); loss = criterion(out, yb) / ACCUM_STEPS
        loss.backward()
        if step % ACCUM_STEPS == 0:
            optimizer.step(); optimizer.zero_grad(set_to_none=True)

In [12]:
import os, json, torch
import numpy as np
from torchvision import datasets, transforms
from torchvision.models import resnet18, resnet50
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Adjust if you saved elsewhere
SAVE_DIR  = "/content/drive/MyDrive/fire_resnet"
BEST_PATH = os.path.join(SAVE_DIR, "best_fire_resnet.pt")

# Use the same size you just trained with
IMG_SIZE = 320

# If DATA_DIR isn't defined in this runtime, set it:
try:
    DATA_DIR
except NameError:
    DATA_DIR = "/content/data_split"  # change if needed

ckpt = torch.load(BEST_PATH, map_location=device)
train_cti = ckpt["class_to_idx"]                      # mapping used during training
class_names = [c for c,_ in sorted(train_cti.items(), key=lambda kv: kv[1])]
norm = ckpt.get("norm", {"mean":[0.485,0.456,0.406], "std":[0.229,0.224,0.225]})

val_tf = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.14)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(norm["mean"], norm["std"]),
])

val_dir = os.path.join(DATA_DIR, "val")
val_ds  = datasets.ImageFolder(val_dir, transform=val_tf)
val_loader = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

# Rebuild the exact model head
arch = ckpt.get("arch","resnet18")
num_classes = len(train_cti)
if arch == "resnet50":
    model = resnet50(num_classes=num_classes)
else:
    model = resnet18(num_classes=num_classes)
model.load_state_dict(ckpt["state_dict"])
model.to(device).eval()

# Run forward pass over val once
all_probs = []
with torch.no_grad():
    for xb, _ in val_loader:
        xb = xb.to(device)
        with torch.amp.autocast('cuda', enabled=(device.type=='cuda')):
            logits = model(xb)
            probs = torch.softmax(logits, dim=1).cpu().numpy()  # columns in train_cti order
        all_probs.append(probs)
all_probs = np.concatenate(all_probs, axis=0)

# Ground truth (names) and file paths in loader order
all_paths = [p for (p, _) in val_ds.samples]
all_true_names = [val_ds.classes[y] for (_, y) in val_ds.samples]
y_true_idx = np.array([train_cti[name] for name in all_true_names])  # map to training indices


In [13]:
pred_idx = all_probs.argmax(1)
n = len(class_names)
cm = np.zeros((n,n), dtype=int)
for t,p in zip(y_true_idx, pred_idx): cm[t,p] += 1

print("Confusion (rows=true, cols=pred) in training-class order:", class_names)
print(cm)

tp = np.diag(cm).astype(float)
prec = tp / np.maximum(cm.sum(0), 1)
rec  = tp / np.maximum(cm.sum(1), 1)
for i, cname in enumerate(class_names):
    print(f"{cname:8s}  precision={prec[i]:.3f}  recall={rec[i]:.3f}")


Confusion (rows=true, cols=pred) in training-class order: ['fire', 'no_fire']
[[157  15]
 [ 28  57]]
fire      precision=0.849  recall=0.913
no_fire   precision=0.792  recall=0.671


In [15]:
# Picking a different threshold to improve no_fire
import numpy as np

fire_idx   = train_cti['fire']
no_fire_idx= train_cti['no_fire']
p_fire = all_probs[:, fire_idx]
y_fire = (y_true_idx == fire_idx).astype(int)   # 1=fire, 0=no_fire

grid = []
for t in np.linspace(0.05, 0.95, 19):
    yhat = (p_fire >= t).astype(int)
    tp = ((y_fire==1)&(yhat==1)).sum()
    fp = ((y_fire==0)&(yhat==1)).sum()
    tn = ((y_fire==0)&(yhat==0)).sum()
    fn = ((y_fire==1)&(yhat==0)).sum()
    prec = tp/max(tp+fp,1); rec=tp/max(tp+fn,1); spec=tn/max(tn+fp,1)  # spec = no_fire recall
    f1 = 2*prec*rec/max(prec+rec,1e-9)
    grid.append((t, prec, rec, spec, f1, fp, fn))
grid = np.array(grid, dtype=object)

# choose by your goal
t_f1     = float(grid[grid[:,4].argmax(), 0])
t_spec80 = float(grid[(grid[:,3] >= 0.80)][0,0]) if (grid[:,3] >= 0.80).any() else 0.5
print("t_f1:", round(t_f1,2), "t_spec80:", round(t_spec80,2))


t_f1: 0.6 t_spec80: 0.8


In [17]:
#Take the current false positives (true=no_fire, predicted=fire) and fold them into training so the model learns those patterns.

import csv, os, shutil
FP_DIR = "/content/hard_negatives"
os.makedirs(FP_DIR, exist_ok=True)

with open("/content/eval/val_predictions.csv") as fh:
    for i,row in enumerate(csv.DictReader(fh)):
        if row["true"]=="no_fire" and row["pred"]=="fire":
            src = row["path"]
            if os.path.exists(src):
                shutil.copy2(src, os.path.join(FP_DIR, os.path.basename(src)))

# copy hard negatives into training/no_fire
TR_NOFIRE = os.path.join(DATA_DIR, "train", "no_fire")
for f in os.listdir(FP_DIR):
    shutil.copy2(os.path.join(FP_DIR,f), os.path.join(TR_NOFIRE, f"hn_{f}"))
print("Added hard negatives to train/no_fire")

Added hard negatives to train/no_fire


In [18]:
#retrain ResNet-18 @ 320 with the regularization you used and a small bias toward no_fire
# class weights from CURRENT train set
import numpy as np, torch, torch.nn as nn
from torch.utils.data import DataLoader, WeightedRandomSampler

counts = np.bincount([y for _,y in train_ds.samples], minlength=len(train_ds.classes))
w_inv = counts.sum() / (len(counts) * counts)  # inverse frequency
class_weights = torch.tensor(w_inv, dtype=torch.float, device=device)

# upweight hard negatives inside the sampler
sample_w = np.ones(len(train_ds.samples), dtype=float)
hn_basenames = set([f"hn_{os.path.basename(p)}" for p,_ in train_ds.samples])  # names we just added
for i,(path,y) in enumerate(train_ds.samples):
    if y == train_ds.class_to_idx['no_fire'] and os.path.basename(path) in hn_basenames:
        sample_w[i] = 3.0  # emphasize hard negatives

sampler = WeightedRandomSampler(sample_w, num_samples=len(sample_w), replacement=True)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=2, pin_memory=True)

# loss + mild head dropout
in_feats = model.fc.in_features
model.fc = nn.Sequential(nn.Dropout(0.2), nn.Linear(in_feats, len(train_ds.classes)))
criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.05)


In [20]:
# === Short retrain (8–10 epochs) + eval + threshold sweep ===
import os, json, time, numpy as np
import torch, torch.nn as nn
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms
from torchvision.models import resnet18, resnet50, ResNet18_Weights, ResNet50_Weights

# --- paths / config ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIR = globals().get("DATA_DIR", "/content/data_split")  # expects train/ and val/ inside
SAVE_DIR = "/content/drive/MyDrive/fire_resnet"              # change if you like
os.makedirs(SAVE_DIR, exist_ok=True)

ARCH        = "resnet18"   # keep rn18 for this quick pass
IMG_SIZE    = 320
BASE_BS     = 64           # what worked at 224
BATCH_SIZE  = max(16, int(BASE_BS * (224/IMG_SIZE)**2))  # ~32 at 320
EPOCHS      = 10
PATIENCE    = 4
LR          = 3e-4
WEIGHT_DEC  = 5e-4
PRETRAINED  = True
MIXED_PREC  = True

# --- data/transforms (augmentation tuned to reduce orange-light FPs) ---
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

train_tf = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(12),
    transforms.ColorJitter(0.35, 0.35, 0.35, 0.10),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.RandomErasing(p=0.25, scale=(0.02, 0.15), ratio=(0.3, 3.3)),
])
val_tf = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.14)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_dir = os.path.join(DATA_DIR, "train")
val_dir   = os.path.join(DATA_DIR, "val")
train_ds  = datasets.ImageFolder(train_dir, transform=train_tf)
val_ds    = datasets.ImageFolder(val_dir,   transform=val_tf)
class_to_idx = train_ds.class_to_idx
idx_to_class = {v:k for k,v in class_to_idx.items()}

# --- class weights (inverse frequency) + sampler (boost hard negatives if prefixed "hn_") ---
counts = np.bincount([y for _,y in train_ds.samples], minlength=len(train_ds.classes))
inv_freq = counts.sum() / (len(counts) * np.maximum(counts, 1))
class_weights_t = torch.tensor(inv_freq, dtype=torch.float, device=device)

sample_w = np.array([inv_freq[y] for _,y in train_ds.samples], dtype=np.float32)
for i,(p,y) in enumerate(train_ds.samples):
    if idx_to_class[y] == 'no_fire' and os.path.basename(p).startswith("hn_"):
        sample_w[i] *= 3.0  # extra emphasis on hard negatives
sampler = WeightedRandomSampler(sample_w, num_samples=len(sample_w), replacement=True)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=max(64, BATCH_SIZE), shuffle=False, num_workers=2, pin_memory=True)

# --- model ---
if ARCH == "resnet50":
    model = resnet50(weights=ResNet50_Weights.DEFAULT if PRETRAINED else None)
    in_feats = model.fc.in_features
else:
    model = resnet18(weights=ResNet18_Weights.DEFAULT if PRETRAINED else None)
    in_feats = model.fc.in_features
model.fc = nn.Sequential(nn.Dropout(0.2), nn.Linear(in_feats, len(class_to_idx)))
model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights_t, label_smoothing=0.05)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DEC)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=2)
scaler = torch.amp.GradScaler('cuda', enabled=(device.type=='cuda' and MIXED_PREC))

def run_epoch(loader, train=True):
    model.train(mode=train)
    tot_loss = tot_acc = n = 0
    for xb, yb in loader:
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        if train:
            optimizer.zero_grad(set_to_none=True)
            if scaler.is_enabled():
                with torch.amp.autocast('cuda'):
                    out = model(xb); loss = criterion(out, yb)
                scaler.scale(loss).backward()
                scaler.step(optimizer); scaler.update()
            else:
                out = model(xb); loss = criterion(out, yb)
                loss.backward(); optimizer.step()
        else:
            with torch.no_grad():
                with torch.amp.autocast('cuda', enabled=scaler.is_enabled()):
                    out = model(xb); loss = criterion(out, yb)
        bs = xb.size(0)
        tot_loss += loss.item()*bs
        tot_acc  += (out.argmax(1) == yb).float().mean().item()*bs
        n += bs
    return tot_loss/n, tot_acc/n

# --- train with early stopping, save best ---
best_val = 0.0
bad_epochs = 0
best_path = os.path.join(SAVE_DIR, "best_fire_resnet.pt")

for epoch in range(1, EPOCHS+1):
    tr_loss, tr_acc = run_epoch(train_loader, train=True)
    va_loss, va_acc = run_epoch(val_loader,   train=False)
    print(f"Epoch {epoch:02d}/{EPOCHS} | train {tr_loss:.4f}/{tr_acc:.3f} | val {va_loss:.4f}/{va_acc:.3f}")
    scheduler.step(va_loss)

    # save per-epoch (optional but resilient)
    torch.save({
        "state_dict": model.state_dict(),
        "arch": ARCH,
        "img_size": IMG_SIZE,
        "class_to_idx": class_to_idx,
        "norm": {"mean": mean, "std": std},
        "epoch": epoch,
        "val_acc": va_acc,
    }, os.path.join(SAVE_DIR, f"ckpt_epoch_{epoch:02d}.pt"))

    if va_acc > best_val:
        best_val = va_acc; bad_epochs = 0
        torch.save({
            "state_dict": model.state_dict(),
            "arch": ARCH,
            "img_size": IMG_SIZE,
            "class_to_idx": class_to_idx,
            "norm": {"mean": mean, "std": std},
            "epoch": epoch,
            "val_acc": va_acc,
        }, best_path)
        with open(os.path.join(SAVE_DIR, "class_to_idx.json"), "w") as f:
            json.dump(class_to_idx, f, indent=2)
        print(f"  Saved new best to {best_path} (val_acc={best_val:.3f})")
    else: # early stopping
        bad_epochs += 1
        if bad_epochs >= PATIENCE:
            print(f"Early stopping after {bad_epochs} epochs without improvement.")
            break

print("Training finished. Best val_acc:", best_val)

Epoch 01/10 | train 0.5630/0.697 | val 0.5616/0.829
  Saved new best to /content/drive/MyDrive/fire_resnet/best_fire_resnet.pt (val_acc=0.829)
Epoch 02/10 | train 0.4265/0.795 | val 0.6760/0.743
Epoch 03/10 | train 0.3642/0.846 | val 0.4601/0.864
  Saved new best to /content/drive/MyDrive/fire_resnet/best_fire_resnet.pt (val_acc=0.864)
Epoch 04/10 | train 0.3844/0.826 | val 0.7238/0.728
Epoch 05/10 | train 0.3604/0.840 | val 0.4107/0.860
Epoch 06/10 | train 0.3092/0.874 | val 0.4576/0.829
Epoch 07/10 | train 0.3006/0.872 | val 0.3970/0.872
  Saved new best to /content/drive/MyDrive/fire_resnet/best_fire_resnet.pt (val_acc=0.872)
Epoch 08/10 | train 0.2818/0.894 | val 0.5436/0.817
Epoch 09/10 | train 0.2577/0.914 | val 1.0183/0.669
Epoch 10/10 | train 0.2504/0.916 | val 0.4556/0.840
Training finished. Best val_acc: 0.8715953307392996


In [23]:
#Resnet 50 @320 with grad accumulation (8–10 epochs)
import os, json, time, numpy as np, torch, torch.nn as nn
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms
from torchvision.models import resnet50, ResNet50_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# paths
DATA_DIR = "/content/data_split"                       # train/ and val/ inside
SAVE_DIR = "/content/drive/MyDrive/fire_resnet_rn50"   # keep RN50 runs separate
os.makedirs(SAVE_DIR, exist_ok=True)

# config
ARCH        = "resnet50"
IMG_SIZE    = 320
BASE_BS     = 64
BATCH_SIZE  = 24                       # safe on T4/L4; adjust if you have more VRAM
ACCUM_STEPS = 2                        # effective batch ~= 48
EPOCHS      = 10
PATIENCE    = 4
LR          = 3e-4
WEIGHT_DEC  = 5e-4
PRETRAINED  = True
MIXED_PREC  = True

# transforms (same philosophy you used to reduce orange-light FPs)
mean = [0.485, 0.456, 0.406]; std = [0.229, 0.224, 0.225]
train_tf = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(12),
    transforms.ColorJitter(0.35,0.35,0.35,0.10),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.RandomErasing(p=0.25, scale=(0.02,0.15), ratio=(0.3,3.3)),
])
val_tf = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.14)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

train_ds = datasets.ImageFolder(os.path.join(DATA_DIR,"train"), transform=train_tf)
val_ds   = datasets.ImageFolder(os.path.join(DATA_DIR,"val"),   transform=val_tf)
class_to_idx = train_ds.class_to_idx
idx_to_class = {v:k for k,v in class_to_idx.items()}

# inverse-frequency class weights + weighted sampler
counts = np.bincount([y for _,y in train_ds.samples], minlength=len(train_ds.classes))
inv_freq = counts.sum() / (len(counts) * np.maximum(counts,1))
class_weights_t = torch.tensor(inv_freq, dtype=torch.float, device=device)

sample_w = np.array([inv_freq[y] for _,y in train_ds.samples], dtype=np.float32)
# if you added hard negatives with prefix "hn_", boost them:
for i,(p,y) in enumerate(train_ds.samples):
    if idx_to_class[y]=='no_fire' and os.path.basename(p).startswith("hn_"):
        sample_w[i] *= 3.0
sampler = WeightedRandomSampler(sample_w, num_samples=len(sample_w), replacement=True)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=max(64,BATCH_SIZE), shuffle=False, num_workers=2, pin_memory=True)

# model
model = resnet50(weights=ResNet50_Weights.DEFAULT if PRETRAINED else None)
in_feats = model.fc.in_features
model.fc = nn.Sequential(nn.Dropout(0.2), nn.Linear(in_feats, len(class_to_idx)))
model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights_t, label_smoothing=0.05)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DEC)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=2)
scaler = torch.amp.GradScaler('cuda', enabled=(device.type=='cuda' and MIXED_PREC))

def train_or_eval(loader, train=True):
    model.train(mode=train)
    tot_loss = tot_acc = n = 0
    if train:
        optimizer.zero_grad(set_to_none=True)
    for step,(xb,yb) in enumerate(loader,1):
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        if train:
            if scaler.is_enabled():
                with torch.amp.autocast('cuda'):
                    out = model(xb); loss = criterion(out, yb)/ACCUM_STEPS
                scaler.scale(loss).backward()
                if step % ACCUM_STEPS == 0:
                    scaler.step(optimizer); scaler.update()
                    optimizer.zero_grad(set_to_none=True)
            else:
                out = model(xb); loss = criterion(out, yb)/ACCUM_STEPS
                loss.backward()
                if step % ACCUM_STEPS == 0:
                    optimizer.step(); optimizer.zero_grad(set_to_none=True)
        else:
            with torch.no_grad():
                with torch.amp.autocast('cuda', enabled=scaler.is_enabled()):
                    out = model(xb); loss = criterion(out, yb)
        bs = xb.size(0)
        tot_loss += loss.item()*bs if not train else (loss.item()*bs*ACCUM_STEPS)
        tot_acc  += (out.argmax(1)==yb).float().mean().item()*bs
        n += bs
    return tot_loss/n, tot_acc/n

best_val, bad = 0.0, 0
best_path = os.path.join(SAVE_DIR, "best_fire_resnet.pt")

for epoch in range(1, EPOCHS+1):
    tr_loss, tr_acc = train_or_eval(train_loader, train=True)
    va_loss, va_acc = train_or_eval(val_loader,   train=False)
    print(f"Epoch {epoch:02d}/{EPOCHS} | train {tr_loss:.4f}/{tr_acc:.3f} | val {va_loss:.4f}/{va_acc:.3f}")
    scheduler.step(va_loss)
    torch.save({"state_dict":model.state_dict(),"arch":"resnet50","img_size":IMG_SIZE,
                "class_to_idx":class_to_idx,"norm":{"mean":mean,"std":std},
                "epoch":epoch,"val_acc":va_acc}, os.path.join(SAVE_DIR,f"ckpt_epoch_{epoch:02d}.pt"))
    if va_acc > best_val:
        best_val, bad = va_acc, 0
        torch.save({"state_dict":model.state_dict(),"arch":"resnet50","img_size":IMG_SIZE,
                    "class_to_idx":class_to_idx,"norm":{"mean":mean,"std":std},
                    "epoch":epoch,"val_acc":va_acc}, best_path)
        with open(os.path.join(SAVE_DIR,"class_to_idx.json"),"w") as f:
            json.dump(class_to_idx,f,indent=2)
        print(f"  Saved best → {best_path} (val_acc={best_val:.3f})")
    else:
        bad += 1
        if bad >= PATIENCE:
            print("Early stopping."); break

print("Best val_acc:", round(best_val,3))


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 151MB/s]


Epoch 01/10 | train 0.4744/0.691 | val 0.4614/0.833
  Saved best → /content/drive/MyDrive/fire_resnet_rn50/best_fire_resnet.pt (val_acc=0.833)
Epoch 02/10 | train 0.3598/0.850 | val 0.5011/0.802
Epoch 03/10 | train 0.3275/0.866 | val 0.4192/0.872
  Saved best → /content/drive/MyDrive/fire_resnet_rn50/best_fire_resnet.pt (val_acc=0.872)
Epoch 04/10 | train 0.3139/0.877 | val 0.4695/0.837
Epoch 05/10 | train 0.2611/0.910 | val 0.5547/0.802
Epoch 06/10 | train 0.2543/0.917 | val 0.4104/0.864
Epoch 07/10 | train 0.2397/0.932 | val 0.4300/0.875
  Saved best → /content/drive/MyDrive/fire_resnet_rn50/best_fire_resnet.pt (val_acc=0.875)
Epoch 08/10 | train 0.2264/0.929 | val 0.4350/0.848
Epoch 09/10 | train 0.1974/0.957 | val 0.5509/0.821
Epoch 10/10 | train 0.1828/0.969 | val 0.4971/0.833
Best val_acc: 0.875


In [25]:
# RN50 eval + threshold
import os, json, numpy as np, torch
from torchvision import datasets, transforms
from torchvision.models import resnet18, resnet50
from torch.utils.data import DataLoader
import torch.nn as nn # Import nn module

device   = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_DIR = "/content/data_split"
SAVE_DIR = "/content/drive/MyDrive/fire_resnet_rn50"
BEST     = f"{SAVE_DIR}/best_fire_resnet.pt"
IMG_SIZE = 320

ckpt = torch.load(BEST, map_location=device)
cti  = ckpt["class_to_idx"]; itc = {v:k for k,v in cti.items()}
norm = ckpt.get("norm", {"mean":[0.485,0.456,0.406], "std":[0.229,0.224,0.225]})
arch = ckpt.get("arch","resnet50")

val_tf = transforms.Compose([
    transforms.Resize(int(IMG_SIZE*1.14)),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(norm["mean"], norm["std"]),
])
val_ds = datasets.ImageFolder(os.path.join(DATA_DIR,"val"), transform=val_tf)
val_loader = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

# rebuild model with the Sequential head
num_classes = len(cti)
if arch == "resnet50":
    model = resnet50(weights=None) # Initialize without pretrained weights
    in_feats = model.fc.in_features
else:
    model = resnet18(weights=None) # Initialize without pretrained weights
    in_feats = model.fc.in_features

# Recreate the Sequential layer with Dropout and Linear
model.fc = nn.Sequential(nn.Dropout(0.2), nn.Linear(in_feats, num_classes))


model.load_state_dict(ckpt["state_dict"]); model.to(device).eval()

# collect probs
probs = []
with torch.no_grad(), torch.amp.autocast('cuda', enabled=(device.type=='cuda')):
    for xb,_ in val_loader:
        xb = xb.to(device)
        logits = model(xb)
        probs.append(torch.softmax(logits, dim=1).cpu().numpy())
probs = np.concatenate(probs, 0)

# ground truth mapped to training indices
y_true = np.array([cti[val_ds.classes[y]] for _,y in val_ds.samples])

# confusion @ argmax
pred = probs.argmax(1)
cm = np.zeros((len(cti),len(cti)), dtype=int)
for t,p in zip(y_true, pred): cm[t,p]+=1
print("Classes:", [itc[i] for i in range(len(cti))])
print("Confusion @ argmax:\n", cm)

# threshold sweep for 'fire'
fire_idx = cti["fire"]; nof_idx = cti["no_fire"]
p_fire = probs[:, fire_idx]; y_fire = (y_true==fire_idx).astype(int)

rows=[]
for t in np.linspace(0.05, 0.95, 19):
    yhat = (p_fire >= t).astype(int)
    tp = ((y_fire==1)&(yhat==1)).sum()
    fp = ((y_fire==0)&(yhat==1)).sum()
    tn = ((y_fire==0)&(yhat==0)).sum()
    fn = ((y_fire==1)&(yhat==0)).sum()
    prec = tp/max(tp+fp,1); rec=tp/max(tp+fn,1); spec=tn/max(tn+fp,1)  # spec = no_fire recall
    f1 = 2*prec*rec/max(prec+rec,1e-9)
    rows.append((t,prec,rec,spec,f1))
grid = np.array(rows, dtype=float)

t_f1     = grid[grid[:,4].argmax(), 0]
t_spec80 = grid[grid[:,3] >= 0.80][0,0] if (grid[:,3] >= 0.80).any() else 0.5
print(f"RN50 thresholds → t_f1={t_f1:.2f}, t_spec80={t_spec80:.2f}")

# confusion @ chosen threshold
THRESH = t_spec80
pred_th = np.where(p_fire >= THRESH, fire_idx, nof_idx)
cm_t = np.zeros_like(cm)
for t,p in zip(y_true, pred_th): cm_t[t,p]+=1
print(f"\nConfusion @ threshold={THRESH:.2f}:\n", cm_t)

# save deploy meta
with open(f"{SAVE_DIR}/deploy_meta.json","w") as f:
    json.dump({"threshold_fire": float(THRESH),
               "img_size": IMG_SIZE, "norm": norm,
               "class_to_idx": cti, "arch": arch}, f, indent=2)
print("Saved", f"{SAVE_DIR}/deploy_meta.json")

Classes: ['fire', 'no_fire']
Confusion @ argmax:
 [[141  31]
 [  1  84]]
RN50 thresholds → t_f1=0.10, t_spec80=0.10

Confusion @ threshold=0.10:
 [[161  11]
 [ 11  74]]
Saved /content/drive/MyDrive/fire_resnet_rn50/deploy_meta.json


In [27]:
# Compare RN18@320 vs RN50@320 with F1- and specificity-driven thresholds
import os, json, numpy as np, torch
from torchvision import datasets, transforms
from torchvision.models import resnet18, resnet50
from torch.utils.data import DataLoader
import torch.nn as nn # Import nn module

def eval_model(save_dir, img_size=320):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ckpt = torch.load(f"{save_dir}/best_fire_resnet.pt", map_location=device)
    cti  = ckpt["class_to_idx"]; itc = {v:k for k,v in cti.items()}
    norm = ckpt.get("norm", {"mean":[0.485,0.456,0.406], "std":[0.229,0.224,0.225]})
    arch = ckpt.get("arch","resnet18")

    tf = transforms.Compose([
        transforms.Resize(int(img_size*1.14)),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(norm["mean"], norm["std"]),
    ])
    val_ds = datasets.ImageFolder("/content/data_split/val", transform=tf)
    loader = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

    # Initialize model
    if arch=="resnet50":
        model = resnet50(weights=None) # Initialize without pretrained weights
        in_feats = model.fc.in_features
    else:
        model = resnet18(weights=None) # Initialize without pretrained weights
        in_feats = model.fc.in_features

    # Recreate the Sequential layer with Dropout and Linear
    model.fc = nn.Sequential(nn.Dropout(0.2), nn.Linear(in_feats, len(cti)))

    model.load_state_dict(ckpt["state_dict"]); model.to(device).eval()

    probs=[]
    with torch.no_grad(), torch.amp.autocast('cuda', enabled=(device.type=='cuda')):
        for xb,_ in loader:
            xb = xb.to(device)
            probs.append(torch.softmax(model(xb), dim=1).cpu().numpy())
    probs = np.concatenate(probs,0)
    y_true = np.array([cti[val_ds.classes[y]] for _,y in val_ds.samples])

    fire = cti["fire"]; nof = cti["no_fire"]
    p = probs[:, fire]; y = (y_true==fire).astype(int)

    # sweep thresholds
    def sweep(p,y):
        best_f1 = (-1,0)   # (score, thr)
        spec80  = None
        for t in np.linspace(0.05,0.95,19):
            yhat = (p>=t).astype(int)
            tp = ((y==1)&(yhat==1)).sum()
            fp = ((y==0)&(yhat==1)).sum()
            tn = ((y==0)&(yhat==0)).sum()
            fn = ((y==1)&(yhat==0)).sum()
            prec = tp/max(tp+fp,1); rec = tp/max(tp+fn,1); spec = tn/max(tn+fp,1)
            f1 = 2*prec*rec/max(prec+rec,1e-9)
            if f1 > best_f1[0]: best_f1 = (f1,t)
            if spec80 is None and spec >= 0.80: spec80 = t
        return best_f1[1], spec80 if spec80 is not None else 0.5

    t_f1, t_spec80 = sweep(p,y)

    # metrics @ those thresholds
    def metrics_at(t):
        pred = np.where(p>=t, fire, nof)
        cm = np.zeros((2,2),dtype=int)
        for t_,p_ in zip((y_true==fire).astype(int), (pred==fire).astype(int)):
            cm[t_,p_] += 1
        tp, fp = cm[1,1], cm[0,1]
        tn, fn = cm[0,0], cm[1,0]
        prec = tp/max(tp+fp,1); rec = tp/max(tp+fn,1); spec = tn/max(tn+fp,1)
        f1 = 2*prec*rec/max(prec+rec,1e-9)
        return dict(threshold=float(t), precision=float(prec), recall_fire=float(rec),
                    recall_no_fire=float(spec), f1=float(f1), cm=cm.tolist())

    return {"dir": save_dir,
            "t_f1": metrics_at(t_f1),
            "t_spec80": metrics_at(t_spec80)}

r18 = eval_model("/content/drive/MyDrive/fire_resnet", img_size=320)
r50 = eval_model("/content/drive/MyDrive/fire_resnet_rn50", img_size=320)

import pandas as pd
df = pd.DataFrame([
    {"model":"RN18@320","mode":"t_f1",**r18["t_f1"]},
    {"model":"RN18@320","mode":"t_spec80",**r18["t_spec80"]},
    {"model":"RN50@320","mode":"t_f1",**r50["t_f1"]},
    {"model":"RN50@320","mode":"t_spec80",**r50["t_spec80"]},
])
print(df[["model","mode","threshold","f1","precision","recall_fire","recall_no_fire"]].to_string(index=False))

   model     mode  threshold       f1  precision  recall_fire  recall_no_fire
RN18@320     t_f1       0.25 0.923077   0.939759     0.906977        0.882353
RN18@320 t_spec80       0.20 0.919540   0.909091     0.930233        0.811765
RN50@320     t_f1       0.10 0.936047   0.936047     0.936047        0.870588
RN50@320 t_spec80       0.10 0.936047   0.936047     0.936047        0.870588


In [30]:
# --- Pick which run you want to plot ---
# RN18:
RUN_DIR = "/content/drive/MyDrive/fire_resnet"

# RN50:
#RUN_DIR = "/content/drive/MyDrive/fire_resnet_rn50"

# --- Common config ---
DATA_DIR = "/content/data_split"   # has train/ and val/
IMG_SIZE = 320

import os, json, glob, math, numpy as np, matplotlib.pyplot as plt
from datetime import datetime

STAMP   = datetime.now().strftime("%Y%m%d_%H%M%S")
OUT_DIR = os.path.join(RUN_DIR, f"figs_{STAMP}")
os.makedirs(OUT_DIR, exist_ok=True)

BEST_PATH = os.path.join(RUN_DIR, "best_fire_resnet.pt")
META_PATH = os.path.join(RUN_DIR, "deploy_meta.json")

if not os.path.exists(BEST_PATH):
    raise FileNotFoundError(f"Checkpoint not found: {BEST_PATH}")
print("Using RUN_DIR:", RUN_DIR)
print("Figures will be saved to:", OUT_DIR)

def savefig_multi(basename, tight=True):
    if tight: plt.tight_layout()
    png = os.path.join(OUT_DIR, f"{basename}.png")
    pdf = os.path.join(OUT_DIR, f"{basename}.pdf")
    svg = os.path.join(OUT_DIR, f"{basename}.svg")
    plt.savefig(png, dpi=300); plt.savefig(pdf); plt.savefig(svg)
    print("Saved:", png, "and .pdf/.svg")


Using RUN_DIR: /content/drive/MyDrive/fire_resnet
Figures will be saved to: /content/drive/MyDrive/fire_resnet/figs_20250909_164905


In [31]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# PICK ONE:
# RUN_DIR = "/content/drive/MyDrive/fire_resnet"        # RN18
RUN_DIR = "/content/drive/MyDrive/fire_resnet_rn50"     # RN50

assert RUN_DIR.startswith("/content/drive/"), "RUN_DIR must be a Drive path"


Mounted at /content/drive


In [32]:
import os, time

STAMP = time.strftime("%Y%m%d_%H%M%S")
OUT_DIR = os.path.join(RUN_DIR, f"figs_{STAMP}")
os.makedirs(OUT_DIR, exist_ok=True)
print("OUT_DIR:", OUT_DIR)

# sanity check: can we write here?
test_txt = os.path.join(OUT_DIR, "write_check.txt")
with open(test_txt, "w") as f:
    f.write("ok")
print("Wrote:", test_txt, "size:", os.path.getsize(test_txt))


OUT_DIR: /content/drive/MyDrive/fire_resnet_rn50/figs_20250909_165222
Wrote: /content/drive/MyDrive/fire_resnet_rn50/figs_20250909_165222/write_check.txt size: 2


In [33]:
import matplotlib.pyplot as plt

def savefig_multi(fig, basename):
    """Save a matplotlib Figure to PNG/PDF/SVG in OUT_DIR."""
    for ext in ("png", "pdf", "svg"):
        path = os.path.join(OUT_DIR, f"{basename}.{ext}")
        fig.savefig(path, dpi=300, bbox_inches="tight")
        if not os.path.exists(path):
            raise RuntimeError(f"Failed to save {path}")
    plt.close(fig)  # free memory
    print(f"Saved {basename} → {OUT_DIR}")

def save_current(basename):
    savefig_multi(plt.gcf(), basename)


In [35]:
# Save all graphs for a run (RN18 or RN50) to Google Drive
!pip -q install scikit-learn

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, json, glob, math, time, numpy as np, matplotlib.pyplot as plt
from datetime import datetime
import torch, torch.nn as nn
from torchvision import datasets, transforms
from torchvision.models import resnet18, resnet50
from torch.utils.data import DataLoader
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, confusion_matrix

#  which trained runs to export figures for
RUN_DIRS = [
  "/content/drive/MyDrive/fire_resnet_rn50",  # RN50
  # "/content/drive/MyDrive/fire_resnet",     # RN18 (uncomment if you want both)
]

DATA_DIR = "/content/data_split"  # contains train/ and val/

def savefig_multi(fig, out_dir, basename):
    os.makedirs(out_dir, exist_ok=True)
    for ext in ("png", "pdf", "svg"):
        path = os.path.join(out_dir, f"{basename}.{ext}")
        fig.savefig(path, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print(f"Saved {basename} -> {out_dir}")

def build_model(arch, n_classes):
    # Initialize base model without the default classifier
    if arch=="resnet50":
        model = resnet50(weights=None) # Initialize without pretrained weights
        in_feats = model.fc.in_features
    else:
        model = resnet18(weights=None) # Initialize without pretrained weights
        in_feats = model.fc.in_features

    # Recreate the Sequential layer with Dropout and Linear as used in training
    model.fc = nn.Sequential(nn.Dropout(0.2), nn.Linear(in_feats, n_classes))
    return model


for RUN_DIR in RUN_DIRS:
    assert RUN_DIR.startswith("/content/drive/"), f"RUN_DIR must be on Drive, got: {RUN_DIR}"
    BEST_PATH = os.path.join(RUN_DIR, "best_fire_resnet.pt")
    META_PATH = os.path.join(RUN_DIR, "deploy_meta.json")
    assert os.path.exists(BEST_PATH), f"Missing checkpoint: {BEST_PATH}"

    STAMP   = time.strftime("%Y%m%d_%H%M%S")
    OUT_DIR = os.path.join(RUN_DIR, f"figs_{STAMP}")
    os.makedirs(OUT_DIR, exist_ok=True)

    print("\n=== Exporting figures for:", RUN_DIR, "→", OUT_DIR)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ckpt   = torch.load(BEST_PATH, map_location=device)

    class_to_idx = ckpt["class_to_idx"]
    idx_to_class = {v:k for k,v in class_to_idx.items()}
    arch  = ckpt.get("arch", "resnet18")
    IMG_SIZE = int(ckpt.get("img_size", 320))

    # normalization + chosen threshold (if present)
    norm = {"mean":[0.485,0.456,0.406], "std":[0.229,0.224,0.225]}
    THRESHOLD = 0.5
    if os.path.exists(META_PATH):
        meta = json.load(open(META_PATH))
        norm = meta.get("norm", norm)
        # Check if 'threshold_fire' exists before accessing it
        if 'threshold_fire' in meta:
            THRESHOLD = float(meta["threshold_fire"])

    # data / loader
    val_tf = transforms.Compose([
        transforms.Resize(int(IMG_SIZE*1.14)),
        transforms.CenterCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(norm["mean"], norm["std"]),
    ])
    val_ds = datasets.ImageFolder(os.path.join(DATA_DIR, "val"), transform=val_tf)
    val_loader = DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

    # model + preds
    model = build_model(arch, len(class_to_idx))
    model.load_state_dict(ckpt["state_dict"])
    model.to(device).eval()

    probs_list, y_list = [], []
    with torch.no_grad(), torch.amp.autocast('cuda', enabled=(device.type=='cuda')):
        for xb, yb in val_loader:
            xb = xb.to(device)
            logits = model(xb)
            probs_list.append(torch.softmax(logits, dim=1).cpu().numpy())
            y_list.append(yb.numpy())
    all_probs = np.concatenate(probs_list, 0)
    y_true    = np.concatenate(y_list, 0)
    paths     = [p for (p, _) in val_ds.samples]

    # save per-image predictions CSV
    import csv
    csv_path = os.path.join(OUT_DIR, "val_predictions.csv")
    with open(csv_path, "w", newline="") as fh:
        w = csv.writer(fh); w.writerow(["path","true","p_fire","p_no_fire"])
        for pth, yi, prob in zip(paths, y_true, all_probs):
            w.writerow([pth, val_ds.classes[yi],
                        f"{prob[class_to_idx['fire']]:.6f}",
                        f"{prob[class_to_idx['no_fire']]:.6f}"])
    print("Saved:", csv_path)

    #    confusion matrices
    # argmax
    y_pred_arg = all_probs.argmax(1)
    cm_arg = confusion_matrix(y_true, y_pred_arg,
                              labels=[class_to_idx['fire'], class_to_idx['no_fire']])

    fig = plt.figure(figsize=(4.6,4))
    plt.imshow(cm_arg, interpolation='nearest')
    plt.xticks([0,1], ['fire','no_fire']); plt.yticks([0,1], ['fire','no_fire'])
    plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion – Argmax")
    for i in range(2):
        for j in range(2):
            plt.text(j, i, str(cm_arg[i,j]), ha="center", va="center")
    savefig_multi(fig, OUT_DIR, "confusion_argmax")

    # thresholded
    p_fire = all_probs[:, class_to_idx['fire']]
    y_pred_thr = np.where(p_fire >= THRESHOLD, class_to_idx['fire'], class_to_idx['no_fire'])
    cm_thr = confusion_matrix(y_true, y_pred_thr,
                              labels=[class_to_idx['fire'], class_to_idx['no_fire']])

    fig = plt.figure(figsize=(4.6,4))
    plt.imshow(cm_thr, interpolation='nearest')
    plt.xticks([0,1], ['fire','no_fire']); plt.yticks([0,1], ['fire','no_fire'])
    plt.xlabel("Predicted"); plt.ylabel("True"); plt.title(f"Confusion – Threshold τ={THRESHOLD:.2f}")
    for i in range(2):
        for j in range(2):
            plt.text(j, i, str(cm_thr[i,j]), ha="center", va="center")
    savefig_multi(fig, OUT_DIR, "confusion_thresholded")

    #  ROC & PR curves
    y_bin = (y_true == class_to_idx['fire']).astype(int)

    from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
    fpr, tpr, _ = roc_curve(y_bin, p_fire)
    roc_auc = auc(fpr, tpr)
    fig = plt.figure(figsize=(5,4))
    plt.plot(fpr, tpr, lw=2, label=f"AUC={roc_auc:.3f}")
    plt.plot([0,1],[0,1], linestyle="--")
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate"); plt.title("ROC (fire vs no_fire)")
    plt.legend()
    savefig_multi(fig, OUT_DIR, "roc_curve")

    prec, rec, _ = precision_recall_curve(y_bin, p_fire)
    ap = average_precision_score(y_bin, p_fire)
    fig = plt.figure(figsize=(5,4))
    plt.plot(rec, prec, lw=2, label=f"AP={ap:.3f}")
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision–Recall (fire)")
    plt.legend()
    savefig_multi(fig, OUT_DIR, "pr_curve")

    # threshold sweeps
    ts = np.linspace(0.05, 0.95, 37)
    rows = []
    for t in ts:
        yhat = (p_fire >= t).astype(int)
        tp = ((y_bin==1)&(yhat==1)).sum()
        fp = ((y_bin==0)&(yhat==1)).sum()
        tn = ((y_bin==0)&(yhat==0)).sum()
        fn = ((y_bin==1)&(yhat==0)).sum()
        precision = tp / max(tp+fp,1)
        recall    = tp / max(tp+fn,1)
        spec      = tn / max(tn+fp,1)  # no_fire recall
        f1        = 2*precision*recall / max(precision+recall,1e-9)
        rows.append((t, precision, recall, spec, f1))
    rows = np.array(rows)

    fig = plt.figure(figsize=(6,4))
    plt.plot(rows[:,0], rows[:,1], label="Precision")
    plt.plot(rows[:,0], rows[:,2], label="Recall (fire)")
    plt.plot(rows[:,0], rows[:,3], label="Recall (no_fire)")
    plt.axvline(THRESHOLD, linestyle="--", label=f"τ={THRESHOLD:.2f}")
    plt.xlabel("Threshold τ on p(fire)"); plt.ylabel("Score"); plt.title("Precision/Recall vs τ")
    plt.legend()
    savefig_multi(fig, OUT_DIR, "threshold_sweep_precision_recall")

    fig = plt.figure(figsize=(6,4))
    plt.plot(rows[:,0], rows[:,4], label="F1 (fire)")
    plt.axvline(THRESHOLD, linestyle="--", label=f"τ={THRESHOLD:.2f}")
    plt.xlabel("Threshold τ on p(fire)"); plt.ylabel("F1"); plt.title("F1 vs τ")
    plt.legend()
    savefig_multi(fig, OUT_DIR, "threshold_sweep_f1")

    # --- training curves from saved checkpoints ---
    ckpts = sorted(glob.glob(os.path.join(RUN_DIR, "ckpt_epoch_*.pt")))
    if len(ckpts) > 0:
        epochs, val_accs, val_losses = [], [], []
        criterion = nn.CrossEntropyLoss()

        # build a loader we can reuse for loss recompute
        # (reuse val_loader created above)
        for pth in ckpts:
            c = torch.load(pth, map_location=device)
            ep = int(os.path.basename(pth).split("_")[-1].split(".")[0])
            epochs.append(ep)
            val_accs.append(float(c.get("val_acc", np.nan)))

            # build model with correct head for loss recompute
            m = build_model(c.get("arch", arch), len(class_to_idx))
            m.load_state_dict(c["state_dict"]); m.to(device).eval()

            tot, n = 0.0, 0
            with torch.no_grad(), torch.amp.autocast('cuda', enabled=(device.type=='cuda')):
                for xb, yb in val_loader:
                    xb, yb = xb.to(device), yb.to(device)
                    logits = m(xb)
                    loss = criterion(logits, yb)
                    tot += float(loss.item()) * xb.size(0)
                    n   += xb.size(0)
            val_losses.append(tot / max(n,1))

        # val acc plot
        fig = plt.figure(figsize=(6,4))
        plt.plot(epochs, val_accs, marker="o")
        plt.xlabel("Epoch"); plt.ylabel("Val Accuracy"); plt.title(f"Validation Accuracy over Epochs ({arch})")
        savefig_multi(fig, OUT_DIR, "training_val_accuracy")

        # val loss plot
        fig = plt.figure(figsize=(6,4))
        plt.plot(epochs, val_losses, marker="o")
        plt.xlabel("Epoch"); plt.ylabel("Val Loss"); plt.title(f"Validation Loss over Epochs ({arch})")
        savefig_multi(fig, OUT_DIR, "training_val_loss")
    else:
        print("No ckpt_epoch_*.pt files found; skipping training curves.")

    # list outputs
    print("\nFiles saved to:", OUT_DIR)
    !ls -lh "$OUT_DIR"

Mounted at /content/drive

=== Exporting figures for: /content/drive/MyDrive/fire_resnet_rn50 → /content/drive/MyDrive/fire_resnet_rn50/figs_20250909_165910
Saved: /content/drive/MyDrive/fire_resnet_rn50/figs_20250909_165910/val_predictions.csv
Saved confusion_argmax -> /content/drive/MyDrive/fire_resnet_rn50/figs_20250909_165910
Saved confusion_thresholded -> /content/drive/MyDrive/fire_resnet_rn50/figs_20250909_165910
Saved roc_curve -> /content/drive/MyDrive/fire_resnet_rn50/figs_20250909_165910
Saved pr_curve -> /content/drive/MyDrive/fire_resnet_rn50/figs_20250909_165910
Saved threshold_sweep_precision_recall -> /content/drive/MyDrive/fire_resnet_rn50/figs_20250909_165910
Saved threshold_sweep_f1 -> /content/drive/MyDrive/fire_resnet_rn50/figs_20250909_165910
Saved training_val_accuracy -> /content/drive/MyDrive/fire_resnet_rn50/figs_20250909_165910
Saved training_val_loss -> /content/drive/MyDrive/fire_resnet_rn50/figs_20250909_165910

Files saved to: /content/drive/MyDrive/fire_